In [78]:
import os
import subprocess
import Bio
from Bio import SeqIO 
from Bio import AlignIO
from Bio.Align import MultipleSeqAlignment
import glob
from Bio import SeqRecord
import pandas as pd
####pip install genomicranges #this worked 
import genomicranges as gr
from Bio.Seq import Seq
####from Bio.SeqRecord import SeqRecord  
import pyranges as pr
from Bio import AlignIO, SeqIO

In [79]:
def make_sbatch_file(filename):
    
    my_list = ["#!/bin/bash", 
               "#SBATCH --job-name=all_bz", 
               "#SBATCH --nodes=1",  
               "#SBATCH --ntasks=1",                     
               "#SBATCH --cpus-per-task=10",              
               "#SBATCH --mem=20gb",                    
               "#SBATCH --partition=20",                
               "##SBATCH --output all_bz-%j.out",  
               "#SBATCH --mail-type=ALL",               
               "#SBATCH --mail-user=hlharris@wi.mit.edu"] 
    
    with open(filename, "w") as file: 
        for item in my_list:
            file.write(item + '\n') 
    
def calc_zeros(alignment): 
    count = 0 
    return [0 if base == "-" else (count := count + 1) for base in alignment[0]]


def crop_alignment(start, end, alignment):
        
    zeros_seq = calc_zeros(alignment)
    #print(zeros_seq) #MIGHT NEED TO SUBTRACT 1 FROM EACH 
    try:
        ix_start = zeros_seq.index(start)
        #print(ix_start)
        ix_end = zeros_seq.index(end) #pretty elegant
        #print(ix_end)
        cropped_alignment = alignment[:, ix_start:ix_end]
        return cropped_alignment
    except: 
        return None
    
    
def concat_alignment(gtf_file, alignment):
   # print(alignment)
    complete_align_type = MultipleSeqAlignment([]) 
    #add the groups to complete align: 
    for recordix in range(len(alignment)): 
        #new_record = SeqRecord.SeqRecord("") 
        new_record = SeqRecord("")
        new_record.id = alignment[recordix].id 
        new_record.seq = Seq("") #added this 

        complete_align_type.append(new_record)
    
   # print(complete_align_type)
    for index, row in gtf_file.iterrows():     
        crop_align = crop_alignment(row[3], row[4], alignment)
        #print(crop_align)
        if crop_align is not None: 
            for recordix in range(len(crop_align)): 
                complete_align_type[recordix].seq += crop_align[recordix].seq #append additional sequence
        
    return complete_align_type

def has_ACGT(sequence):
    count_ACGT = sum(c in 'ACTG' for c in sequence)
    return count_ACGT > len(sequence) / 1000 #true or false

In [6]:
#generate alignment of full gene sequences

gene = 'ZFX'

tree = '(humanmasked_X humanmasked_Y (opossum))'
    
    
subprocess.run(["cp", '/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/multiz_7sp/'+ 'ZFX' + "/" + 'humanmasked', '/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/multiz_dog_bull/opossum/zfx_y_human_opossum/humanmasked_X'])
subprocess.run(["cp", '/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/multiz_7sp/'+ 'ZFY' + "/" + 'humanmasked', '/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/multiz_dog_bull/opossum/zfx_y_human_opossum/humanmasked_Y'])
subprocess.run(["cp", '/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/multiz_dog_bull/'+ 'opossum' + "/" + 'ZFX_fullgene', '/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/multiz_dog_bull/opossum/zfx_y_human_opossum/opossum']) 


subprocess.run(["touch", "all_bz.log"])  #making a new file 
    
with open("all_bz.log", "w") as log_file:
    result = subprocess.run(['all_bz', '-', tree], stdout=log_file, stderr=subprocess.PIPE, check=True)
        
make_sbatch_file("testj1.sh") #make sbatch file - it makes a new one each time through so you dont have to delete the old one 
print(gene) 
with open("testj1.sh", "a") as file: #append to the file 
    file.write("bash ") 
    file.write("all_bz.log" + "\n")
    file.write("tba '" + tree + "' *.*.maf tba.maf >&tba.log" + "\n") 
    file.write("maf_project tba.maf humanmasked_X '" + tree + "' > human_proj.maf" + "\n") #project from the perspective of the X chr 
    file.write("msa_view -o FASTA human_proj.maf > " + gene + "_msa.fa")
        
subprocess.run(["sbatch", 'testj1.sh'], stderr=subprocess.PIPE)
  

ZFX
Submitted batch job 1479188


CompletedProcess(args=['sbatch', 'testj1.sh'], returncode=0, stderr=b'')

In [ ]:
#FOR ALIGNMENTS */- 
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

genes = ['ZFX']

regions = ["exon", "intron"] #, "promoter"]  
#regions = [ "promoter"] #just do promoter  


list_of_species = ['humanmasked_X', 'humanmasked_Y', 'opossum']

files_generated = [] 

spec1 = 'humanmasked_X' 

for gene in genes: 
    print(gene) 

    #change dir: 
    
    gtf_file = pd.read_csv("/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/" + gene + "_gtf_all103023.txt", delimiter="\t", header = None) #pd.read_csv("/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/" + gene + "_gtf_all0721.txt", delimiter="\t", header = None) 
    #print(gtf_file)
    
#    for spec2ix in range(len(list_of_species)): 
       # print(spec2ix, 'im at the top')
        
#        spec2 = list_of_species[spec2ix] 
#        print(spec2, 'spec2')
#        spec2ix = spec2ix + 1
#        second_align_file_name = spec1 + "." + spec2  
        
#        msa = list(SeqIO.parse('/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/multiz_dog_bull/' + gene + "/human_chicken/" + gene + "_msa.fa", "fasta")) #get the MASKED version there

#        new_filepath = "/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/multiz_dog_bull/" + gene + "/human_chicken/" + second_align_file_name + "_msa.fa"
#        pair_alignment = MultipleSeqAlignment([msa[0]]) #add human alignment

#        pair_alignment.append(msa[spec2ix]) 

#        SeqIO.write(pair_alignment, new_filepath, "fasta") #as fasta

  
    # separate into multiple files based on GTF file 
    for region in regions: 
        print(region, "region")

        gtf_file1 = gtf_file[gtf_file[2] == region] 
       
        new_alignment = AlignIO.read("/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/multiz_dog_bull/opossum/zfx_y_human_opossum/ZFX_msa.fa", "fasta")
        #print(gtf_file1)
        returned_align = concat_alignment(gtf_file1,new_alignment) 
        #print(returned_align[0].seq)
        filtered_sequences = [record for record in returned_align if has_ACGT(record.seq)] 
        
        filtered_sequences[0].id = 'humanX'
        filtered_sequences[1].id = 'humanY'
        filtered_sequences[2].id = 'op'
        
        if filtered_sequences:
            SeqIO.write(filtered_sequences, "/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/multiz_dog_bull/opossum/zfx_y_human_opossum/ZFX_"  + region + "_msa.phy", "phylip") #as phylip
                
            new_alignment = MultipleSeqAlignment([])

            for record in filtered_sequences:
                print(record)
    # Replace '*' with '-' in the sequence
                modified_seq = str(record.seq).replace('*', '-')
    # Replace 'N' with '-'
                modified_seq = modified_seq.replace('N', '-')    
    # Create a new SeqRecord with the modified sequence 
                new_record = SeqRecord(Seq(modified_seq), id=record.id)
    
    # Append the new record to the new alignment
                new_alignment.append(new_record)
        
                SeqIO.write(new_alignment, "/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/multiz_dog_bull/opossum/zfx_y_human_opossum/ZFX_"  + region + "_msa.phy", "phylip") #as phylip

            subprocess.run(['/lab/page_scratch/hannah/trimal/source/trimal', '-in', '/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/multiz_dog_bull/opossum/zfx_y_human_opossum/ZFX_'  + region + "_msa.phy", '-out', '/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/multiz_dog_bull/opossum/zfx_y_human_opossum/ZFX_' + region +  '_msa_filtered.phy', '-gt', '1'],  stderr=subprocess.PIPE)

        else: 
            print("stupid check") 

In [76]:
#from alignment get places where X and Y have different bases 
#if X is same as opossum, add value depending on what the change is 
#if Y is the same as opossum, add value depending on what the change is 
  
#first value for X, second for Y
dict_types = {'CG_AT': [0,0], 'CG_GC':[0,0], 'CG_TA':[0,0], 'TA_AT':[0,0], 'TA_CG':[0,0], 'TA_GC':[0,0]}     
    

#alignment = AlignIO.read('/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/multiz_dog_bull/opossum/zfx_y_human_opossum/ZFX_promoter_msa_filtered.phy', 'phylip')

#alignment = AlignIO.read('/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/multiz_dog_bull/opossum/zfx_y_human_opossum/ZFX_exon_msa_filtered.phy', 'phylip')

alignment = AlignIO.read('/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/multiz_dog_bull/opossum/zfx_y_human_opossum/ZFX_intron_msa_filtered.phy', 'phylip')


for i in range(alignment.get_alignment_length()):
    # Get the characters at position i for each sequence
    characters = alignment[:, i]
    X = characters[0]
    Y = characters[1]
    op = characters[2]
    if X == Y or X == Y == op:
        next
   
    if X == op: 
        valueix = 1
        
        if X == "C":
            if Y == "A":
                dict_types['CG_AT'][valueix] += 1
            if Y == "T":
                dict_types['CG_TA'][valueix] += 1
            if Y == "G":
                dict_types['CG_GC'][valueix] += 1
            
        if X == "G":
            if Y == "A":
                dict_types['CG_TA'][valueix] += 1
            if Y == "T":
                dict_types['CG_AT'][valueix] += 1
            if Y == "C":
                dict_types['CG_GC'][valueix] += 1
        
        if X == "T":
            if Y == "A":
                dict_types['TA_AT'][valueix] += 1
            if Y == "G":
                dict_types['TA_GC'][valueix] += 1
            if Y == "C":
                dict_types['TA_CG'][valueix] += 1
                
        
        if X == "A":
            if Y == "T":
                dict_types['TA_AT'][valueix] += 1
            if Y == "G":
                dict_types['TA_CG'][valueix] += 1
            if Y == "C":
                dict_types['TA_GC'][valueix] += 1
        
                
    if Y == op: 
        valueix = 0
               
        if Y == "C":
            if X == "A":
                dict_types['CG_AT'][valueix] += 1
            if X == "T":
                dict_types['CG_TA'][valueix] += 1
            if X == "G":
                dict_types['CG_GC'][valueix] += 1
            
        if Y == "G":
            if X == "A":
                dict_types['CG_TA'][valueix] += 1
            if X == "T":
                dict_types['CG_AT'][valueix] += 1
            if X == "C":
                dict_types['CG_GC'][valueix] += 1
        
        if Y == "T":
            if X  == "A":
                dict_types['TA_AT'][valueix] += 1
            if X  == "G":
                dict_types['TA_GC'][valueix] += 1
            if X  == "C":
                dict_types['TA_CG'][valueix] += 1
        
        
        if Y == "A":
            if X  == "T":
                dict_types['TA_AT'][valueix] += 1
            if X  == "G":
                dict_types['TA_CG'][valueix] += 1
            if X  == "C":
                dict_types['TA_GC'][valueix] += 1

    
    
df = pd.DataFrame.from_dict(dict_types, orient='index', columns=['X', 'Y'])


In [77]:
df
#df.to_csv('mutation_types.txt')

,X,Y
CG_AT,2,6
CG_GC,1,1
CG_TA,5,8
TA_AT,3,9
TA_CG,8,12
TA_GC,4,2


In [66]:
#df.to_csv('mutation_types_exon.txt')

In [69]:
#df.to_csv('mutation_types_intron.txt')

In [80]:
#df.to_csv('mutation_promoter.txt')

In [98]:
#count up C+ As
from Bio.SeqUtils import GC
alignment = AlignIO.read('/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/multiz_dog_bull/opossum/zfx_y_human_opossum/ZFX_intron_msa_filtered.phy', 'phylip')
alignment_op = alignment[2]

In [99]:
GC(alignment_op.seq)

31.147540983606557

In [100]:
len(alignment_op.seq) * (GC(alignment_op.seq)/100)

95.0

In [101]:
len(alignment_op.seq) * (1- GC(alignment_op.seq)/100)

210.0